In [ ]:
!pip install labelme tensorflow opencv-python matplotlib albumentations numpy

In [ ]:
import cv2
import os
import time
import uuid
import csv
import tensorflow as tf



ImgPath = os.path.join('Data', 'CollectedImages')
no_img = 1
Lcap = cv2.VideoCapture(0)  # captures feed from cam, camera configured as "0"
Lcap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', '4', 'v'))  # video codec for good video feed
Lcap.set(3, 900)  # video resolution
Lcap.set(4, 600)
for Img_no in range(no_img):
    print('capturing image(s) {}'.format(Img_no))
    Img_id = os.path.join(ImgPath, f'{str(uuid.uuid1())}.jpg')
    ret, frame = Lcap.read()
    cv2.imwrite(Img_id, frame)
    cv2.imshow('Frame', frame)
    
    key = cv2.waitKey(10)  # ms waittime for closing window
    if key == 27:  # key 'esc' for closing camera feed
        break
Lcap.release()
cv2.destroyAllWindows()

In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf




In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
#gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
Img = tf.data.Dataset.list_files('Data\\CollectedImages\\*.jpg', shuffle=False)

In [ ]:
Img.as_numpy_iterator().next()

In [ ]:
def LDAImg(x): 
    encImg = tf.io.read_file(x)
    dImg = tf.io.decode_jpeg(encImg)
    return dImg

In [ ]:
Img = Img.map(LDAImg)

In [ ]:
print(Img.as_numpy_iterator().next())

In [ ]:
type(Img)

In [1]:
#Img = tf.data.Dataset.list_files('Data\\CollectedImages\\*.jpg',shuffle=False)
#print(Img.as_numpy_iterator().next())

#def LDAImg(x):
 #   encImg = tf.io.read_file(x)
  #  dImg = tf.io.decode_jpeg(encImg)
   # return dImg
#Img = Img.map(LDAImg)
#print(Img.as_numpy_iterator().next())
#print(type(Img))
print("ye")


ye


In [ ]:
ImgGen = Img.batch(4).as_numpy_iterator()

In [ ]:
pImg = ImgGen.next()

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(pImg):
    ax[idx].imshow(image) 
plt.show()

In [ ]:
for folder in ['Test','Train','Validation']:
    for file in os.listdir(os.path.join('Data',folder,'CollectedImages')):
        FName = file.split('.')[0]+'.csv'
        ExFPath = os.path.join('Data','Labels',FName)
        if os.path.exists(ExFPath):
            NFPath = os.path.join('Data',folder,'Labels',FName)
            os.replace(ExFPath, NFPath)

In [ ]:
import albumentations as alb

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

In [ ]:
dImg = cv2.imread(os.path.join('Data','Train','CollectedImages','7683b27d-d628-11ed-a79f-ebebb2e11f69.jpg'))

In [ ]:
dImg

In [ ]:
with open(os.path.join('Data','Train','Labels','7683b27d-d628-11ed-a79f-ebebb2e11f69.json'), 'r') as f:
    LImg = list(csv.reader(f))
    

In [ ]:
LImg['shapes'][0]['points']

In [ ]:
Coords = [0,0,0,0]

Coords[0] = LImg['shapes'][0]['points'][0][0]
Coords[1] = LImg['shapes'][0]['points'][0][1]
Coords[2] = LImg['shapes'][0]['points'][1][0]
Coords[3] = LImg['shapes'][0]['points'][1][1]

In [ ]:
Coords


In [ ]:
Coords = list(np.divide(Coords, [800,600,800,600]))

In [ ]:
Coords

In [ ]:
augmented = augmentor(image=dImg, bboxes=[Coords], class_labels=['Face'])

In [2]:
print("h")

h


In [ ]:
augmented['image']

In [ ]:
augmented

In [ ]:
cv2.rectangle(augmented['image'], 
              tuple(np.multiply(augmented['bboxes'][0][:2], [450,450]).astype(int)),
              tuple(np.multiply(augmented['bboxes'][0][2:], [450,450]).astype(int)), 
                    (255,0,0), 2)

plt.imshow(augmented['image'])

In [ ]:
for partition in ['Train','Test','Validation']: 
    for image in os.listdir(os.path.join('Data', partition, 'CollectedImages')):
        dImg = cv2.imread(os.path.join('Data', partition, 'CollectedImages', image))

        Coords = [0,0,0.00001,0.00001]
        LPath = os.path.join('Data', partition, 'Labels', f'{image.split(".")[0]}.json')
        if os.path.exists(LPath):
            with open(LPath, 'r') as f:
                LImg = json.load(f)

            Coords[0] = LImg['shapes'][0]['points'][0][0]
            Coords[1] = LImg['shapes'][0]['points'][0][1]
            Coords[2] = LImg['shapes'][0]['points'][1][0]
            Coords[3] = LImg['shapes'][0]['points'][1][1]
            Coords = list(np.divide(Coords, [800,600,800,600]))

        try: 
            for x in range(150):
                augmented = augmentor(image=dImg, bboxes=[Coords], class_labels=['Face'])
                cv2.imwrite(os.path.join('AugData', partition, 'CollectedImages', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image
                if os.path.exists(LPath):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 
                with open(os.path.join('AugData', partition, 'Labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)
        except Exception as e:
            print(e)

In [ ]:
TrainImg = tf.data.Dataset.list_files('AugData\\Train\\CollectedImages\\*.jpg', shuffle=False)
TrainImg = TrainImg.map(LDAImg)
TrainImg = TrainImg.map(lambda x: tf.image.resize(x, (120,120)))
TrainImg = TrainImg.map(lambda x: x/255)

In [ ]:
TestImg = tf.data.Dataset.list_files('AugData\\Test\\CollectedImages\\*.jpg', shuffle=False)
TestImg = TestImg.map(LDAImg)
TestImg = TestImg.map(lambda x: tf.image.resize(x, (120,120)))
TestImg = TestImg.map(lambda x: x/255)


In [ ]:
ValidationImg = tf.data.Dataset.list_files('AugData\\Validation\\CollectedImages\\*.jpg', shuffle=False)
ValidationImg = ValidationImg.map(LDAImg)
ValidationImg = ValidationImg.map(lambda x: tf.image.resize(x, (120,120)))
ValidationImg = ValidationImg.map(lambda x: x/255)

In [ ]:
TrainImg.as_numpy_iterator().next()

In [ ]:
print ye

In [ ]:
def load_labels(LPath):
    with open(LPath.numpy(), 'r', encoding = "utf-8") as f:
        LImg = json.load(f)
        
    return [LImg['class']], LImg['bbox']

In [ ]:
TrainLabels = tf.data.Dataset.list_files('AugData\\Train\\Labels\\*.json', shuffle=False)
TrainLabels = TrainLabels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
TestLabels = tf.data.Dataset.list_files('AugData\\Test\\Labels\\*.json', shuffle=False)
TestLabels = TestLabels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
ValidationLabels = tf.data.Dataset.list_files('AugData\\Validation\\Labels\\*.json', shuffle=False)
ValidationLabels = ValidationLabels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
ValidationLabels.as_numpy_iterator().next()

In [ ]:
len(TrainImg),len(TrainLabels),len(TestImg),len(TestLabels),len(ValidationImg),len(ValidationLabels)

In [ ]:
Train = tf.data.Dataset.zip((TrainImg, TrainLabels))
Train = Train.shuffle(17000)
Train = Train.batch(8)
Train = Train.prefetch(4)

In [ ]:
Test = tf.data.Dataset.zip((TestImg, TestLabels))
Test = Test.shuffle(5000)
Test = Test.batch(8)
Test = Test.prefetch(4)

In [ ]:
Validation = tf.data.Dataset.zip((ValidationImg, ValidationLabels))
Validation = Validation.shuffle(5000)
Validation = Validation.batch(8)
Validation = Validation.prefetch(4)

In [ ]:
Train.as_numpy_iterator().next()[0]

In [ ]:
DSamp = Train.as_numpy_iterator()

In [ ]:
res = DSamp.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    SampImg = res[0][idx]
    Sampcoords = res[1][1][idx]
    
    cv2.rectangle(SampImg, 
                  tuple(np.multiply(Sampcoords[:2], [120,120]).astype(int)),
                  tuple(np.multiply(Sampcoords[2:], [120,120]).astype(int)), 
                        (255,0,0), 2)

    ax[idx].imshow(SampImg)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
VGG = VGG16(include_top=False)

In [ ]:
VGG.summary()

In [ ]:
def build_model(): 
    ILayer = Input(shape=(120,120,3))
    
    VGG = VGG16(include_top=False)(ILayer)

    
    f1 = GlobalMaxPooling2D()(VGG)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(1, activation='sigmoid')(class1)
    
    
    f2 = GlobalMaxPooling2D()(VGG)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    Facetracker = Model(inputs=ILayer, outputs=[class2, regress2])
    return Facetracker

In [ ]:
Facetracker = build_model()

In [ ]:
Facetracker.summary()

In [ ]:
X, y = Train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
classes, Coords = Facetracker.predict(X)#kerenel crashing

In [ ]:
classes, Coords

In [ ]:
len(Train)

In [ ]:
batches_per_epoch = len(Train)
lr_decay = (1./0.75 -1)/batches_per_epoch

In [ ]:
lr_decay

In [ ]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, decay=lr_decay)

In [ ]:
def localization_loss(y_true, yhat):            
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))
                  
    h_true = y_true[:,3] - y_true[:,1] 
    w_true = y_true[:,2] - y_true[:,0] 

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss

In [ ]:
localization_loss(y[1], Coords)

In [ ]:
classloss(y[0], classes)

In [ ]:
regressloss(y[1], Coords)

In [ ]:
class FaceTracker(Model): 
    def __init__(self, Facetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = Facetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, Coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), Coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, Coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), Coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [ ]:
model = FaceTracker(Facetracker)

In [ ]:
model.compile(opt, classloss, regressloss)

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(Train, epochs=20, validation_data=Validation, callbacks=[tensorboard_callback])

In [ ]:
hist.history

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

In [ ]:
test_data = Test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()

In [ ]:
yhat = Facetracker.predict(test_sample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    Sampcoords = yhat[1][idx]
    
    if yhat[0][idx] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(Sampcoords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(Sampcoords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#Facetracker.save('Facetrackerv2.h5')

In [ ]:
facetracker = load_model('facetrackerv1.h5')

In [ ]:
Lcap = cv2.VideoCapture(0)
while Lcap.isOpened():
    _ , frame = Lcap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = Facetracker.predict(np.expand_dims(resized/255,0))
    Sampcoords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(Sampcoords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(Sampcoords[2:], [450,450]).astype(int)), 
                            (255,202,90), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(Sampcoords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(Sampcoords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,202,90), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'Face', tuple(np.add(np.multiply(Sampcoords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('FaceTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
Lcap.release()
cv2.destroyAllWindows()

In [ ]:
print()